Get the Dataset

In [1]:
import torch 
from torch.utils.data import DataLoader
from torchvision import models, transforms
import cv2
import numpy as np
import json

In [2]:
class KeypointsDataset():
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [3]:
train_dataset = KeypointsDataset('data/images','data/data_train.json')
valid_dataset = KeypointsDataset('data/images','data/data_val.json')

In [4]:
train_dataset = DataLoader(train_dataset, shuffle=True, batch_size=8)
valid_dataset = DataLoader(valid_dataset, shuffle=True, batch_size=8)

In [5]:
model = models.resnet50(pretrained=True)
#model.fc = model.fc.Linear(model.fc.in_features, 28)

c:\Users\hanna m\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hanna m\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model.fc = torch.nn.Linear(model.fc.in_features, 28)

In [7]:
criteria = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [8]:
epochs = 5
for epoch in range(0, epochs):
    for i, (img, kps) in enumerate(train_dataset):
        optimizer.zero_grad()
        output = model(img)
        loss =criteria(output, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14316.275390625
Epoch 0, iter 10, loss: 11326.9912109375
Epoch 0, iter 20, loss: 8161.62255859375
Epoch 0, iter 30, loss: 5733.28271484375
Epoch 0, iter 40, loss: 3694.283935546875
Epoch 0, iter 50, loss: 1976.740966796875
Epoch 0, iter 60, loss: 1111.9639892578125
Epoch 0, iter 70, loss: 392.9232482910156
Epoch 0, iter 80, loss: 181.24867248535156
Epoch 0, iter 90, loss: 78.27155303955078
Epoch 0, iter 100, loss: 57.310848236083984
Epoch 0, iter 110, loss: 38.72127151489258
Epoch 0, iter 120, loss: 38.37017059326172
Epoch 0, iter 130, loss: 35.43738555908203
Epoch 0, iter 140, loss: 28.6087646484375
Epoch 0, iter 150, loss: 43.508445739746094
Epoch 0, iter 160, loss: 29.25193214416504
Epoch 0, iter 170, loss: 42.831153869628906
Epoch 0, iter 180, loss: 24.249330520629883
Epoch 0, iter 190, loss: 40.33829879760742
Epoch 0, iter 200, loss: 26.60870361328125
Epoch 0, iter 210, loss: 60.26813888549805
Epoch 0, iter 220, loss: 37.907833099365234
Epoch 0, iter 230, lo

In [9]:
torch.save(model.state_dict(), 'model_det.pth')

In [10]:
from ultralytics import YOLO